In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.dates import DateFormatter, HourLocator
from scipy.stats import skew, kurtosis,jarque_bera

# Disable Warnings
import warnings
warnings.filterwarnings('ignore')
# Allow Multiple Output per Cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'
# StatsModels for Ordinary Least Squares Regresssion
import statsmodels.api as sm
# Import the adfuller (ADF) Stationarity Test
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.ar_model import AutoReg
# Import QuantStats for Trading Strategy Tear-Sheets
import quantstats as qs

In [2]:
# All data from binance or binance futures || Base currency is USDT

df_btc = pd.read_csv('../data/candle_bns_btc_usdt_1m_202403011651.csv')
df_eth = pd.read_csv('../data/candle_bns_eth_usdt_1m_202403011652.csv')

df_btc_perp = pd.read_csv('../data/candle_bnf_btc_usdt_perpetual_1m_202403011649.csv') 
df_eth_perp = pd.read_csv('../data/candle_bnf_eth_usdt_perpetual_1m_202403011650.csv')

# Sort dataframes by 'timestamp'
df_btc = df_btc.sort_values(by='timestamp')
df_eth = df_eth.sort_values(by='timestamp')
df_btc_perp = df_btc_perp.sort_values(by='timestamp')
df_eth_perp = df_eth_perp.sort_values(by='timestamp')

### Convert to daily data

In [3]:
def convert_to_daily_data(df_btc):
    # Convert 'timestamp' column to datetime
    df_btc['timestamp'] = pd.to_datetime(df_btc['timestamp'])

    # Group by date and aggregate the data
    daily_df_btc = df_btc.groupby(df_btc['timestamp'].dt.date).agg({
        'open': 'first',
        'close': 'last',
        'low': 'min',
        'high': 'max',
        'volume': 'sum',
        'cont_records': 'sum',
        'quote_volume': 'sum'
    }).reset_index()

    return daily_df_btc

daily_df_btc = convert_to_daily_data(df_btc)
daily_df_eth = convert_to_daily_data(df_eth)
daily_df_btc_perp = convert_to_daily_data(df_btc_perp)
daily_df_eth_perp = convert_to_daily_data(df_eth_perp)

daily_df_btc["name"] = "BTC_SPOT"
daily_df_eth["name"] = "ETH_SPOT"
daily_df_btc_perp["name"] = "BTC_PERPETUAL"
daily_df_eth_perp["name"] = "ETH_PERPETUAL"

### First moment analysis, normality check + fourth moment analysis

In [4]:
# Calculate mean, skewness, and kurtosis for each dataset
for df in [daily_df_btc, daily_df_eth, daily_df_btc_perp, daily_df_eth_perp]:
    print(f"Dataset: {df.name.unique()}")
    print("Mean:")
    print(df['close'].mean())
    print("\nSkewness:")
    print(df['close'].skew())
    print("\nKurtosis:")
    print(df['close'].kurtosis())
    print("\n")

for df in [daily_df_btc, daily_df_eth, daily_df_btc_perp, daily_df_eth_perp]:
    print(f"Dataset: {df.name.unique()}")
    jb_test = jarque_bera(df['close'])
    print("Jarque-Bera test statistic:", jb_test[0])
    print("Jarque-Bera p-value:", jb_test[1])
    if jb_test[1] < 0.05:
        print("The data is not normally distributed.")
    else:
        print("The data is normally distributed.")
    print("\n")

# All datasets are NOT normally distributed

Dataset: ['BTC_SPOT']
Mean:
31345.59925058548

Skewness:
1.0027697111482898

Kurtosis:
0.6330500371953001


Dataset: ['ETH_SPOT']
Mean:
1903.7139578454335

Skewness:
1.3747229685168965

Kurtosis:
2.624291749367412


Dataset: ['BTC_PERPETUAL']
Mean:
42083.96592592593

Skewness:
0.7145741964454869

Kurtosis:
1.086627156004464


Dataset: ['ETH_PERPETUAL']
Mean:
2289.644666666667

Skewness:
0.8206992257863454

Kurtosis:
0.9201267054939177


Dataset: ['BTC_SPOT']
Jarque-Bera test statistic: 77.7150411995042
Jarque-Bera p-value: 1.3316595963298414e-17
The data is not normally distributed.


Dataset: ['ETH_SPOT']
Jarque-Bera test statistic: 251.94686584324933
Jarque-Bera p-value: 1.9517905715066655e-55
The data is not normally distributed.


Dataset: ['BTC_PERPETUAL']
Jarque-Bera test statistic: 16.88937505830935
Jarque-Bera p-value: 0.00021503977882605767
The data is not normally distributed.


Dataset: ['ETH_PERPETUAL']
Jarque-Bera test statistic: 18.809469342578513
Jarque-Bera p-value: 8.2

### Carry Calculation

In [5]:
datasets = {
    'BTC': (daily_df_btc, daily_df_btc_perp),
    'ETH': (daily_df_eth, daily_df_eth_perp)
}

for asset, (spot_df, perp_df) in datasets.items():
    spot_df = spot_df.set_index('timestamp')
    perp_df = perp_df.set_index('timestamp')

    # Perform an inner join to align the timestamps
    joined_df = spot_df.join(perp_df['close'], rsuffix='_perp', how='inner')

    # Calculate carry as the difference between spot and perpetual prices
    carry_df = joined_df['close'] - joined_df['close_perp']

    print(f"Asset: {asset}")

    # Calculate the first moment (mean)
    mean_carry = carry_df.mean()
    print(f"Mean of carry: {mean_carry}")

    # Calculate the second moment (variance)
    variance_carry = carry_df.var()
    print(f"Variance of carry: {variance_carry}")

    # Calculate the third moment (skewness)
    skewness_carry = skew(carry_df.dropna())
    print(f"Skewness of carry: {skewness_carry}")

    # Calculate the fourth moment (kurtosis)
    kurtosis_carry = kurtosis(carry_df.dropna())
    print(f"Kurtosis of carry: {kurtosis_carry}")

    print("\n")

Asset: BTC
Mean of carry: -11.440444444445058
Variance of carry: 1083.9012221890364
Skewness of carry: -5.471010441385693
Kurtosis of carry: 45.32471578013937


Asset: ETH
Mean of carry: -0.7266666666666458
Variance of carry: 2.273813432835858
Skewness of carry: -4.061752720129048
Kurtosis of carry: 31.306500220906855




In [6]:
for asset, (spot_df, perp_df) in datasets.items():
    spot_df = spot_df.set_index('timestamp')
    perp_df = perp_df.set_index('timestamp')

    # Perform an inner join to align the timestamps
    joined_df = spot_df.join(perp_df['close'], rsuffix='_perp', how='inner')

    # Calculate carry as the difference between spot and perpetual prices
    carry_df = joined_df['close'] - joined_df['close_perp']

    print(f"Asset: {asset}")

    # Linear Regression Model
    X = sm.add_constant(spot_df['close'])  # Add a constant for the intercept
    X = X.reindex(carry_df.index)  # Reindex X to match carry_df's index
    model = sm.OLS(carry_df, X).fit()
    print(f"Linear Regression Model Summary:")
    print(model.summary())
    print("\n")

    # Autoregressive (AR) Model
    ar_model = AutoReg(carry_df, lags=1).fit()  # Fit an AR(1) model
    print(f"Autoregressive (AR) Model Summary:")
    print(ar_model.summary())
    print("\n")

Asset: BTC
Linear Regression Model Summary:
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.216
Model:                            OLS   Adj. R-squared:                  0.210
Method:                 Least Squares   F-statistic:                     36.71
Date:                Sat, 16 Mar 2024   Prob (F-statistic):           1.32e-08
Time:                        09:57:36   Log-Likelihood:                -646.32
No. Observations:                 135   AIC:                             1297.
Df Residuals:                     133   BIC:                             1302.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const   

/Users/mukeshwaranbaskaran/anaconda3/envs/nb_research/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/mukeshwaranbaskaran/anaconda3/envs/nb_research/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


### Crypto Carry Equation
### f_i_t,T - s_i_t = r_t,T - r*_t,T + δ_t,T + ε_i_t

### Where:

### f_i_t,T is the log futures price for a crypto asset from t to T
### s_i_t is the log spot price
### r_t,T and r*_t,T are the USD and crypto risk-free rates, respectively
### δ_t,T is the crypto convenience yield
### ε_i_t is an exchange-specific pricing error

In [7]:
for asset, (spot_df, perp_df) in datasets.items():
    spot_df = spot_df.set_index('timestamp')
    perp_df = perp_df.set_index('timestamp')

    # Perform an inner join to align the timestamps
    joined_df = spot_df.join(perp_df['close'], rsuffix='_perp', how='inner')

    # Calculate log prices
    spot_df['log_price'] = np.log(spot_df['close'])
    perp_df['log_price'] = np.log(perp_df['close'])

    # Assume constant risk-free rates for USD and crypto
    r_t_T = 0.03  # USD risk-free rate (e.g., 3%)
    r_star_t_T = 0.05  # Crypto risk-free rate (e.g., 5%)

    # Calculate the left-hand side of the equation
    joined_df['lhs'] = perp_df['log_price'] - spot_df['log_price']

    # Calculate the right-hand side of the equation
    joined_df['rhs'] = r_t_T - r_star_t_T + joined_df['lhs'].shift(1)

    # Drop rows with missing or invalid values
    joined_df = joined_df.dropna(subset=['lhs', 'rhs'])

    # Fit a linear regression model
    model = sm.OLS(joined_df['lhs'], sm.add_constant(joined_df['rhs'])).fit()
    print(f"Asset: {asset}")
    print(f"Crypto Carry Equation Model Summary:")
    print(model.summary())
    print("\n")

Asset: BTC
Crypto Carry Equation Model Summary:
                            OLS Regression Results                            
Dep. Variable:                    lhs   R-squared:                       0.222
Model:                            OLS   Adj. R-squared:                  0.216
Method:                 Least Squares   F-statistic:                     37.64
Date:                Sat, 16 Mar 2024   Prob (F-statistic):           9.23e-09
Time:                        09:57:37   Log-Likelihood:                 811.90
No. Observations:                 134   AIC:                            -1620.
Df Residuals:                     132   BIC:                            -1614.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
cons

### Fama regressions --> decompose carry into spot price changes and futures premium changes.


In [8]:
for asset, (spot_df, perp_df) in datasets.items():
    spot_df = spot_df.set_index('timestamp')
    perp_df = perp_df.set_index('timestamp')

    # Perform an inner join to align the timestamps
    joined_df = spot_df.join(perp_df['close'], rsuffix='_perp', how='inner')

    # Calculate log prices
    joined_df['log_price'] = np.log(joined_df['close'])
    joined_df['log_price_perp'] = np.log(joined_df['close_perp'])

    # Calculate log returns
    joined_df['log_ret'] = joined_df['log_price'].diff()
    joined_df['log_ret_perp'] = joined_df['log_price_perp'].diff()

    # Drop the first row (NaN for log returns)
    joined_df = joined_df.dropna(subset=['log_ret', 'log_ret_perp'], how='any').iloc[1:]

    # Calculate carry
    joined_df['carry'] = joined_df['log_price_perp'] - joined_df['log_price']

    # Drop rows with missing or invalid values
    joined_df = joined_df.dropna(subset=['carry', 'log_ret'])

    # Fama regression: decompose carry into spot price changes and futures premium changes
    X = sm.add_constant(joined_df['log_ret'])
    model = sm.OLS(joined_df['carry'].shift(-1), X).fit()

    print(f"Asset: {asset}")
    print(f"Fama Regression Summary:")
    print(model.summary())
    print("\n")

Asset: BTC
Fama Regression Summary:
                            OLS Regression Results                            
Dep. Variable:                  carry   R-squared:                         nan
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Sat, 16 Mar 2024   Prob (F-statistic):                nan
Time:                        09:57:37   Log-Likelihood:                    nan
No. Observations:                 133   AIC:                               nan
Df Residuals:                     131   BIC:                               nan
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const           

### Investor positioning analysis

In [9]:
# Regression of carry on net futures positions of different trader groups
cot_model = sm.OLS.from_formula('carry ~ dealer_pos + leveraged_pos + nonreportable_pos + other_pos', data=cot_df).fit()

NameError: name 'cot_df' is not defined

### Trend-following and attention analysis

In [ ]:
# Regression of carry on social media metrics and past returns
trend_model = sm.OLS.from_formula('carry ~ reddit_subs + past_return_1w + past_return_1m', data=combined_df).fit()

### Risk and return analysis of the cash and carry strategy

In [ ]:
# Calculate cash and carry strategy returns
combined_df['carry_returns'] = np.where(combined_df['carry'] > 0,
                                        combined_df['spot_change'] - combined_df['futures_change'],
                                        combined_df['futures_change'] - combined_df['spot_change'])

# Calculate Sharpe Ratio
carry_sharpe_ratio = combined_df['carry_returns'].mean() / combined_df['carry_returns'].std() * np.sqrt(252)

# Regression of contract liquidations on lagged carry
liquidations_model = sm.OLS.from_formula('liquidations ~ carry.shift(1)', data=combined_df).fit()

### Crypto carry and crash risk analysis
### Investigate the link between crypto carry and crash risk using option market data.

In [ ]:
# Regression of implied skewness on carry
skew_model = sm.OLS.from_formula('implied_skew ~ carry', data=combined_df).fit()

# Regression of realized skewness on lagged carry
realized_skew_model = sm.OLS.from_formula('realized_skew ~ carry.shift(1)', data=combined_df).fit()

### Visualization

In [ ]:
# Time-series plot of carry
plt.figure(figsize=(12, 6))
plt.plot(daily_df_btc['timestamp'], daily_df_btc['carry'])
plt.title('Bitcoin Carry over Time')
plt.xlabel('Date')
plt.ylabel('Carry')
plt.show()

# Scatter plot of carry against Reddit subscribers
plt.figure(figsize=(8, 6))
plt.scatter(daily_df_btc['reddit_subs'], daily_df_btc['carry'])
plt.title('Bitcoin Carry vs. Reddit Subscribers')
plt.xlabel('Reddit Subscribers')
plt.ylabel('Carry')
plt.show()

# Plot of regression coefficients
coef_plot = pd.Series(trend_model.params)
coef_plot.plot(kind='bar', yerr=trend_model.bse, capsize=4)
plt.title('Regression Coefficients: Carry on Trend and Attention Variables')
plt.xlabel('Variable')
plt.ylabel('Coefficient')
plt.show()